In [11]:
import pandas as pd
import os
import numpy as np
os.chdir(r"C:/Users/abasa/OneDrive/Робочий стіл/Rework/DataCamp/lesson_3")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

def answer_one():
    

    file_energy= "energy_indicators.xls"
    energy= pd.read_excel(file_energy,skiprows=16,skipfooter=38)
    energy = energy.drop(energy.columns[[0, 1]],axis = 1)
    energy=energy.rename(columns={energy.columns[0]: "Country", energy.columns[1]: "Energy Supply", energy.columns[2]: "Energy Supply per Capita",energy.columns[3]: "% Renewable"})
    
    energy= energy.replace('...',np.NaN)
    energy['Energy Supply'] = energy['Energy Supply'][1:].apply(lambda x: x*1000000)
    def remove_digits(data):
        newData = ''.join([i for i in data if not i.isdigit()])
        i = newData.find('(')
        if i>-1: newData = newData[:i]
        return newData.strip()
    energy['Country'] = energy['Country'][1:].apply(remove_digits)
    energy['Country'] = energy['Country'].replace('Republic of Korea','South Korea')
    energy['Country'] = energy['Country'].replace('United States of America','United States')
    energy['Country'] = energy['Country'].replace('United Kingdom of Great Britain and Northern Ireland','United Kingdom')
    energy['Country'] = energy['Country'].replace('China, Hong Kong Special Administrative Region','Hong Kong')
    energy['Country'] = energy['Country'].replace('Bolivia (Plurinational State of)','Bolivia')
    energy['Country'] = energy['Country'].replace('Switzerland','Switzerland')
    
    #print(energy)
    
    file_gdp= "world_bank.csv"
    GDP=pd.read_csv(file_gdp,skiprows=4)
    
    GDP.rename(columns={'Country Name': 'Country'}, inplace=True)
    GDP.replace({"Korea, Rep.": "South Korea", 
                "Iran, Islamic Rep.": "Iran",
                "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
    #print(GDP["Country"])
    
    ScimEn=pd.read_excel("scima.xlsx")
    
    result=pd.merge(pd.merge(energy, GDP, on='Country'), ScimEn, on='Country')
    result=result.set_index('Country')
    result=result[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    result = (result.loc[result['Rank'].isin([i for i in range(1, 16)])])
    #result=result.loc[result['Rank']<15]
    result.sort_values('Rank',inplace=True)
    return result
def answer_two():
    Top15 = answer_one()
    years = ['2006', '2007', '2008',
           '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    Top15["avgGDP"] = Top15[years].mean(axis=1)
    return Top15.sort_values("avgGDP",ascending=False)["avgGDP"]
def answer_three():
    Top15 = answer_one()
    Top15["AvgGDP"] = answer_two()
    Top15.sort_values("AvgGDP", ascending=False, inplace=True)
    gdp_2015 = Top15.iloc[5]['2015']
    gdp_2006 = Top15.iloc[5]['2006']
    
    return abs(gdp_2015-gdp_2006)
def answer_four():
    Top15 = answer_one()
    Top15['Ratio']=Top15['Self-citations']/Top15['Citations']

    return (Top15.index[Top15['Ratio'].argmax()],Top15['Ratio'].max())
def answer_five():
    Top15 = answer_one()
    Top15['Population']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15=Top15.sort_values("Population",ascending=False)
    
    return Top15.index[2]
def answer_six():
    Top15 = answer_one()
    Top15['Population']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Citable documents per Capita']=Top15['Citable documents']/Top15['Population']
    
    return Top15["Citable documents per Capita"].corr(Top15['Energy Supply per Capita'])
def answer_seven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    continents = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
    Top15['Estimate Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    for continent, frame in Top15.groupby(ContinentDict):
        continents.loc[continent] = [len(frame), frame['Estimate Population'].sum(),frame['Estimate Population'].mean(),frame['Estimate Population'].std()]
    return continents

answer_one()



,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,303064,301778,3036531,2092737,10.02,273,1.271910e+11,93,19.75491,2.752132e+12,3.550343e+12,4.594307e+12,5.101703e+12,6.087164e+12,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13
United States,2,184851,181106,2623922,819242,14.19,389,9.083800e+10,286,11.57098,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,60257,58589,590570,213760,9.80,198,3.319500e+10,26,14.96908,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Japan,4,52780,52281,557023,132113,10.55,203,1.898400e+10,149,10.23282,4.601663e+12,4.579751e+12,5.106679e+12,5.289493e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,47141,45928,748994,132737,15.89,244,7.920000e+09,124,10.60047,2.717060e+12,3.106182e+12,2.938882e+12,2.425798e+12,2.491110e+12,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12
Germany,6,42343,41464,528645,115432,12.48,211,1.326100e+10,165,17.90153,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,39424,39189,142937,67935,3.63,96,3.070900e+10,214,17.28868,9.899305e+11,1.299706e+12,1.660846e+12,1.222644e+12,1.524917e+12,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12
Canada,8,35588,34940,665415,113363,18.70,246,1.043100e+10,296,61.94543,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,31260,29959,433388,105641,13.86,177,6.530000e+09,109,33.66723,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12
